# GFS Forecast Maps from Thredds Server via NCSS and Siphon
# MSLP w/ Highs and Lows and Thickness

## Justin Richling
## 11/07/19
* Included updates to THREDDS catalog heirarchy

https://doi.org/10.6084/m9.figshare.5244637.v1

In [1]:
# System Tools
import os

# Importing Datetime Libraries
from datetime import datetime, timedelta

# CartoPy Map Plotting Libraires
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Numerical and Scientific Libraries
import numpy as np
from scipy.ndimage import gaussian_filter

# Accessing Data from XLM Catalog via Siphon Libraries
from siphon.catalog import TDSCatalog
from siphon.ncss import NCSS

# MetPy Libraries
from metpy.plots import add_metpy_logo

# NetCDF Libraries
from netCDF4 import num2date

# Matplotlib Plotting Libraries
import matplotlib.pyplot as plt
from matplotlib import patheffects

# Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Set the font 
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 18,
        }

## Helper Functions

In [3]:
# Thanks to the crew over at Metpy for this handy little function
def find_time_var(var, time_basename='time'):
    for coord_name in var.coordinates.split():
        if coord_name.startswith(time_basename):
            return coord_name
    raise ValueError('No time variable found for ' + var.name)

In [4]:
# MetPy Function
def plot_maxmin_points(AX,lon, lat, data, extrema, nsize, symbol, color='k',
                       plotValue=True, transform=None):
    """
    This function will find and plot relative maximum and minimum for a 2D grid. The function
    can be used to plot an H for maximum values (e.g., High pressure) and an L for minimum
    values (e.g., low pressue). It is best to used filetered data to obtain  a synoptic scale
    max/min value. The symbol text can be set to a string value and optionally the color of the
    symbol and any plotted value can be set with the parameter color
    lon = plotting longitude values (2D)
    lat = plotting latitude values (2D)
    data = 2D data that you wish to plot the max/min symbol placement
    extrema = Either a value of max for Maximum Values or min for Minimum Values
    nsize = Size of the grid box to filter the max and min values to plot a reasonable number
    symbol = String to be placed at location of max/min value
    color = String matplotlib colorname to plot the symbol (and numerica value, if plotted)
    plot_value = Boolean (True/False) of whether to plot the numeric value of max/min point
    The max/min symbol will be plotted on the current axes within the bounding frame
    (e.g., clip_on=True)
    """
    from scipy.ndimage.filters import maximum_filter, minimum_filter

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')
    
    mxy, mxx = np.where(data_ext == data)
    #print(mxy,mxx)

    for i in range(len(mxy)):
        #ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], symbol, color=color, size=24,
        #        clip_on=True, horizontalalignment='center', verticalalignment='center',
        #        transform=transform)
        #ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]],
        #        '\n' + str(np.int(data[mxy[i], mxx[i]])),
        #        color=color, size=12, clip_on=True, fontweight='bold',
        #        horizontalalignment='center', verticalalignment='top', transform=transform)
        outline_effect = [patheffects.withStroke(linewidth=2.5, foreground='black')]
        
        A = AX.text(lon[mxx[i]], lat[mxy[i]], symbol, color=color, size=24,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)
        A.set_path_effects(outline_effect)
        
        B = AX.text(lon[mxx[i]], lat[mxy[i]]-0.66,
                str(np.int(data[mxy[i], mxx[i]])),
                color=color, size=12, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform)
        B.set_path_effects(outline_effect)
        
        
        #AX.text(lon[mxx[i]], lat[mxy[i]],
        #        '\n' + str(np.int(data[mxy[i], mxx[i]])),
        #        color=color, size=12, clip_on=True, fontweight='bold',
        #        horizontalalignment='center', verticalalignment='top', transform=transform)
        
        #text_time = ax.text(.995, 0.01, 
        #    Time,
        #    horizontalalignment='right', transform=ax.transAxes,
        #    color='white', fontsize=12, weight='bold',zorder=15)

        #text_time2 = ax.text(0.005, 0.01, 
        #            "NWS Radar (dbz)",
        #            horizontalalignment='left', transform=ax.transAxes,
        #            color='white', fontsize=12, weight='bold',zorder=15)

        #outline_effect = [patheffects.withStroke(linewidth=5, foreground='black')]
        #a.set_path_effects(outline_effect)
        #b.set_path_effects(outline_effect)

<h2>----------------------------------------------//---------------------------------------------------------</h2>

## Set the Map Projection

In [5]:
# Set Projection of Data
datacrs = ccrs.PlateCarree()

# Set Projection of Plot
plotcrs = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)

# Add Map Features
states_provinces = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_1_states_provinces_lakes',scale='50m', facecolor='none')

country_borders = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_0_countries',scale='50m', facecolor='none')

# Colorbar Axis Placement (under figure)
colorbar_axis = [0.183, 0.09, 0.659, 0.03] # [left, bottom, width, height]

# Lat/Lon Extents [lon0,lon1,lat0,lat1]
extent = [-130., -70, 20., 60.]

<h2>----------------------------------------------//---------------------------------------------------------</h2>

## Figure out where the saved maps will go

In [6]:
now = datetime.utcnow()
#now = datetime(2019,4,10,0,0)
today_day = int('{0:%d}'.format(now))
today_year = int('{0:%Y}'.format(now))
today_month = int('{0:%m}'.format(now))
print(today_day,today_year,today_month)


# Set a path to save the plots with string format for the date to set the month and day
im_save_path ="/path/to/saved/images/"
im_save_path ="/Users/chowdahead/Desktop/Weather_Blog/{}/{}_{}/".format(today_year,today_month,today_day)
print(im_save_path)

# Check to see if the folder already exists, if not create it
if not os.path.isdir(im_save_path):
    os.makedirs(im_save_path)

# Uncomment if you want to automatically change to the map folder    
#os.chdir(im_save_path)

12 2019 11
/Users/chowdahead/Desktop/Weather_Blog/2019/11_12/


<h2>----------------------------------------------//---------------------------------------------------------</h2>
<h2>----------------------------------------------//---------------------------------------------------------</h2>

<h1><font style="font-size:32px"><center>-- Plotting all of the GFS forecast hours for the current day --</center></font></h1>

<h2><font><center>-- Highs/Lows and 1000-500mb Thickness --</center></font></h2>

<h2>----------------------------------------------//---------------------------------------------------------</h2>

In [17]:
# Request the GFS data from the thredds server
gfs = TDSCatalog('http://thredds-jetstream.unidata.ucar.edu/thredds/catalog/grib/'
                 'NCEP/GFS/Global_0p5deg/catalog.xml')

dataset = list(gfs.datasets.values())[1]
#print(dataset.access_urls)

# Create NCSS object to access the NetcdfSubset
ncss = NCSS(dataset.access_urls['NetcdfSubset'])

# get current date and time
#now = forecast_times[0]
now = datetime(today_year,today_month,today_day,0,0)
# define time range you want the data for
start = now
print(start)
delt_t = 48
end = now + timedelta(hours=delt_t)

query = ncss.query()
query.time_range(start, end)
query.lonlat_box(north=60, south=20, east=310, west=230)
query.accept('netcdf4')
query.variables("MSLP_Eta_model_reduction_msl","Geopotential_height_isobaric")


# Request data for the variables you want to use
data = ncss.get_data(query)

# Pull out the lat and lon data
lats = data.variables['lat'][:]
lons = data.variables['lon'][:]

# Get time into a datetime object
time_var = data.variables[find_time_var(data.variables['MSLP_Eta_model_reduction_msl'])]
time_var = num2date(time_var[:], time_var.units).tolist()
time_strings = [t.strftime('%m/%d %H:%M') for t in time_var]

# Combine 1D latitude and longitudes into a 2D grid of locations
lon_2d, lat_2d = np.meshgrid(lons, lats)

time_var = data.variables[find_time_var(data.variables["MSLP_Eta_model_reduction_msl"])]
time_final = num2date(time_var[:].squeeze(), time_var.units)

mslp_name = "MSLP_Eta_model_reduction_msl"
hgt_name = "Geopotential_height_isobaric"

mslp = data.variables[mslp_name][:]/100
hgt = data.variables[hgt_name][:]

print("done.")

2019-11-12 00:00:00
done.


In [31]:
def HiLo_Thck(time_index):
    time = time_strings[time_index]
    
    kw_clabels = {'fontsize': 11, 'inline': True, 'inline_spacing': 5, 'fmt': '%i',
              'rightside_up': True, 'use_clabeltext': True}
    
    fig = plt.figure(figsize=(17., 11.))

    add_metpy_logo(fig, 25, 925, size='small')

# Add the map 
    ax = plt.subplot(111, projection=plotcrs)

# Set extent and plot map lines
    ax.set_extent(extent, datacrs)
    
    ax.coastlines(resolution='50m')

    import cartopy.feature as cfeature
    import cartopy.io.shapereader as shpreader
    political_boundaries = cfeature.NaturalEarthFeature(category='cultural',
                                   name='admin_0_boundary_lines_land',
                                   scale='50m', facecolor='none')
    
    state_borders = cfeature.NaturalEarthFeature(
                category='cultural', name='admin_1_states_provinces_lines',
                scale='50m', facecolor='none')
    ax.add_feature(state_borders, edgecolor='b', linewidth=1, zorder=3)
    country_borders = cfeature.NaturalEarthFeature(category='cultural',
                name='admin_0_countries',scale='50m', facecolor='none')
    ax.add_feature(country_borders,edgecolor='b',linewidth=0.2)
    
# Add state/country boundaries to plot
    ax.add_feature(cfeature.STATES)
    ax.add_feature(cfeature.BORDERS)
    
    
    file_time = str(time_final[i]).replace("-","_")
    file_time = file_time.replace(" ","_")
    file_time = file_time.replace(":","")
    file_time = file_time[:-2]
    file_time = file_time+"Z"
    
    
    title_time = "{}".format(today_year)+'-'+time_strings[i].replace("/","-")[:-5]
    print(file_time)

    init_hour = time_strings[0].replace("/","-")[-5:]+"Z"
    forecast_hour = time_strings[i][-5:]+"Z"
    
    hgt_500 = hgt[time_strings.index(time),data.variables["isobaric6"][:].tolist().index(50000),:,:]
    hgt_1000 = hgt[time_strings.index(time),data.variables["isobaric6"][:].tolist().index(100000),:,:]
    thickness_1000_500 = gaussian_filter(hgt_500 - hgt_1000, sigma=3.0)

# Plot Title
    ax.set_title('GFS: MSLP w/ Highs/Lows, 1000-500mb Thickness (m)'+'\n'+title_time, size=16, loc='left',fontdict=font)
    ax.set_title('Init Hour: '+str(init_hour)+'\nForecast Hour: '+str(forecast_hour), size=16, 
                 loc='right',fontdict=font)

    
                                            # 1000-500mb Thickness
#---------------------------------------------------------------------------------------------------
    clevs = (np.arange(0, 5400, 60), np.arange(5400,5401,1), np.arange(5460, 7000, 60))
    colors = ('tab:blue', 'b', 'tab:red')
    
    for clevthick, color in zip(clevs, colors):
        cs = ax.contour(lons, lats, thickness_1000_500, levels=clevthick, colors=color,
                    linewidths=1.0, linestyles='dashed', transform=datacrs)
        plt.clabel(cs, **kw_clabels)

                                            # MSLP
#---------------------------------------------------------------------------------------------------
    clevmslp = np.arange(800., 1120., 4)
    cs2 = ax.contour(lons, lats, mslp[time_strings.index(time),:,:], clevmslp, colors='k', linewidths=1.25,
                 linestyles='solid', transform=datacrs)
    plt.clabel(cs2, **kw_clabels)

                                            # High and Low Symbols
#---------------------------------------------------------------------------------------------------
    plot_maxmin_points(ax,lons, lats, mslp[time_strings.index(time),:,:], 'max', 50, symbol='H', color='b',  transform=datacrs)
    plot_maxmin_points(ax,lons, lats, mslp[time_strings.index(time),:,:], 'min', 25, symbol='L', color='r', transform=datacrs)
    
    GFS_HILO = im_save_path+"GFS/HILO/"
    if not os.path.isdir(GFS_HILO):
        os.makedirs(GFS_HILO)
    outfile=GFS_HILO+"HL_Thickness_Model_"+file_time+".png"
    fig.savefig(outfile,bbox_inches='tight',dpi=120)
    plt.close(fig)

In [32]:
# PV_Flux(time_index,PV=None,UpFlux=None,MSLP=None,mb500=None)
    
for i in range(0,int(delt_t/3)):
    HiLo_Thck(i)
    print("next.")
print("All done.")

2019_11_12_0000Z
next.
2019_11_12_0300Z
next.
2019_11_12_0600Z
next.
2019_11_12_0900Z
next.
2019_11_12_1200Z
next.
2019_11_12_1500Z
next.
2019_11_12_1800Z
next.
2019_11_12_2100Z
next.
2019_11_13_0000Z
next.
2019_11_13_0300Z
next.
2019_11_13_0600Z
next.
2019_11_13_0900Z
next.
2019_11_13_1200Z
next.
2019_11_13_1500Z
next.
2019_11_13_1800Z
next.
2019_11_13_2100Z
next.
All done.
